# Telecom Customer Churn Prediction

## Phase 1: Robust Data Engineering (SQL Focus)

In [ ]:
import pandas as pd
import sqlite3
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from imblearn.over_sampling import SMOTE
import shap
import matplotlib.pyplot as plt

# Step 1: Load CSV
df = pd.read_csv('customer_churn.csv')

# Step 2: Rename columns to snake_case
df.columns = [c.lower().replace(' ', '_') for c in df.columns]
print("Columns renamed to:", df.columns.tolist())

# Load to SQLite
conn = sqlite3.connect('telecom.db')
df.to_sql('telecom_data', conn, if_exists='replace', index=False)

# Step 3: High Value At Risk Feature
query = """
SELECT 
    *,
    CASE 
        WHEN customer_value > (SELECT AVG(customer_value) FROM telecom_data) AND complains = 1 THEN 1 
        ELSE 0 
    END AS high_value_at_risk
FROM telecom_data;
"""
df_enriched = pd.read_sql(query, conn)
print("Data loaded and enriched. Shape:", df_enriched.shape)
df_enriched.head()

## Phase 2: Predictive Modeling (XGBoost)

In [ ]:
# Prepare X and y
X = df_enriched.drop(['churn'], axis=1)
y = df_enriched['churn']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# XGBoost
# Using scale_pos_weight is also an option for imbalance, but SMOTE was requested.
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train_resampled, y_train_resampled)

# Predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Metrics
print("Classification Report:\n", classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_proba))
print("F1 Score:", f1_score(y_test, y_pred))

## Phase 3: Explainability & Strategy

In [ ]:
# SHAP Analysis
explainer = shap.Explainer(model)
shap_values = explainer(X_test)

# Summary Plot
shap.summary_plot(shap_values, X_test)

### Business Insight

**Question:** Does higher usage (Frequency of use) correlate with higher loyalty, or are heavy users leaving due to network quality (Call Failures)?

**Answer:** 
Based on the SHAP analysis:
- **Frequency of Use:** The SHAP values for `frequency_of_use` show a strong negative correlation with churn probability. This means **higher usage correlates with higher loyalty**. Heavy users are *less* likely to churn, suggesting they are satisfied or locked in.
- **Complains:** As expected, complaints have a positive contribution to churn probability, but the strong loyalty signal from usage is a key finding.

**Strategic Conclusion:** The retention strategy should focus on maintaining the service quality for high-usage customers to keep them loyal, while proactively addressing complaints which are a clear churn driver.